# 7. Dense Neural Networks

In [ ]:
from util import get_image_by_index, decode_class
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import VarianceScaling

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:
# Load the .npz file
data = np.load('dataset_features.npz')

# List all arrays within the .npz file
print(data.files)

# Access individual arrays by their names
X_train = data['trainset_features']
y_train = data['trainset_labels']

X_val = data['validset_features']
y_val = data['validset_labels']

X_test = data['testset_features']
y_test = data['testset_labels']

class_labels = data['class_labels']

# 1-layer model

In [ ]:
model_1layer = Sequential()
model_1layer.add(Dense(6, activation="softmax", input_dim=X_train.shape[1],
                kernel_initializer=VarianceScaling(scale=1.0, seed=0)))
model_1layer.summary()

In [ ]:
# Define loss function, optimizer and metrics to track during training
model_1layer.compile(optimizer="sgd",
              loss="categorical_crossentropy",
              metrics=["acc"])

Data preparation

In [ ]:
X_crossval = np.concatenate((X_train, X_val), axis=0)
y_crossval = np.concatenate((y_train, y_val), axis=0)

In [ ]:
# Rescale train and validation data
scaler = StandardScaler()
X_crossval_preprocessed = scaler.fit_transform(X_crossval)
X_test_preprocessed = scaler.transform(X_test)

## Training

In [ ]:
# Fit model
history = model_1layer.fit(
    x=X_crossval_preprocessed,
    y=y_crossval,
    validation_split=0.2,
    batch_size=32,
    epochs=25,
    shuffle=True,  # Shuffle training samples
)

In [ ]:
# Create two plots: one for the loss value, one for the accuracy
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Plot accuracy values
ax1.plot(history.history["loss"], label="train loss")
ax1.plot(history.history["val_loss"], label="val loss")
ax1.set_title(
    "Validation loss {:.3f} (mean last 3)".format(
        np.mean(history.history["val_loss"][-3:])  # last three values
    )
)
ax1.set_xlabel("epoch")
ax1.set_ylabel("loss value")
ax1.legend()

# Plot accuracy values
ax2.plot(history.history["acc"], label="train acc")
ax2.plot(history.history["val_acc"], label="val acc")
ax2.set_title(
    "Validation accuracy {:.3f} (mean last 3)".format(
        np.mean(history.history["val_acc"][-3:])  # last three values
    )
)
ax2.set_xlabel("epoch")
ax2.set_ylabel("accuracy")
ax2.legend()
plt.show()

In [ ]:
print('Model Accuracy:')
loss, accuracy = model_1layer.evaluate(X_train, y_train, verbose=0)
print(f'On train set: {accuracy:.3f}')
loss, accuracy = model_1layer.evaluate(X_val, y_val, verbose=0)
print(f'On valid set: {accuracy:.3f}')
loss, accuracy = model_1layer.evaluate(X_test, y_test, verbose=0)
print(f'On test  set: {accuracy:.3f}')

# 2-layer model

In [ ]:
y_train.shape

In [ ]:
model_2layer = Sequential()
model_2layer.add(Dense(10, activation="relu", input_dim=X_train.shape[1],
                kernel_initializer=VarianceScaling(scale=2.0, seed=0)))
model_2layer.add(Dense(6, activation="softmax", 
                kernel_initializer=VarianceScaling(scale=1.0, seed=0)))
model_2layer.summary()

In [ ]:
from tensorflow.keras import optimizers

# Define loss function, optimizer and metrics to track during training
model_2layer.compile(optimizer="sgd",
                     loss="categorical_crossentropy",
                     metrics=["acc"])

In [ ]:
# Fit model
history = model_2layer.fit(x=X_crossval_preprocessed,
                           y=y_crossval,
                           validation_split=0.2,
                           batch_size=32,
                           epochs=50,
                           shuffle=True,  # Shuffle training samples
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create two plots: one for the loss value, one for the accuracy
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Plot accuracy values
ax1.plot(history.history["loss"], label="train loss")
ax1.plot(history.history["val_loss"], label="val loss")
ax1.set_title('Validation loss {np.mean(history.history["val_loss"][-3:]:.3f} (mean last 3)')
ax1.set_xlabel("epoch")
ax1.set_ylabel("loss value")
ax1.legend()

# Plot accuracy values
ax2.plot(history.history["acc"], label="train acc")
ax2.plot(history.history["val_acc"], label="val acc")
ax2.set_title(f'Validation accuracy {np.mean(history.history["val_acc"][-3:]):.3f} (mean last 3)')
ax2.set_xlabel("epoch")
ax2.set_ylabel("accuracy")
ax2.legend()
plt.show()

In [ ]:
print('Model Accuracy:')
loss, accuracy = model_2layer.evaluate(X_train, y_train, verbose=0)
print(f'On train set: {accuracy:.3f}')
loss, accuracy = model_2layer.evaluate(X_val, y_val, verbose=0)
print(f'On valid set: {accuracy:.3f}')
loss, accuracy = model_2layer.evaluate(X_test, y_test, verbose=0)
print(f'On test  set: {accuracy:.3f}')